<a href="https://colab.research.google.com/github/100jy/dacon_ts_forecasting/blob/main/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torchcontrib

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset
import torch.optim.adam
from torchcontrib.optim import SWA
import datetime
import matplotlib.pyplot as plt 
from tqdm import tqdm

In [ ]:

train = pd.read_csv("./drive/MyDrive/데이콘/train.csv", encoding = 'euc-kr')

# 시간 관련 변수들
train['DateTime'] = pd.to_datetime(train.DateTime)
#일자
train['Date'] = train.DateTime.dt.date

# 요일 혹은 분기정보
train['DayOfWeek'] = (train.DateTime.dt.weekday)/6
train['DayOfMon'] = ((train.DateTime).dt.day)/31
train['Quarter'] = ((train.DateTime).dt.quarter)/4

train['Year'] = ((train.DateTime.dt.year) -2019)
train['Days'] = (train.DateTime.max() - train.DateTime).dt.days + 1


left = train.iloc[:,:5].groupby(train['Date']).sum().reset_index()
right = train.iloc[:,5:].groupby(train['Date']).mean().reset_index()
train  = pd.merge(left, right, on='Date')

def log_trans(x):
  return np.log(1+x)

train['Days'] = log_trans(train['Days'])


# ts feature 생성 
for target in ['사용자', '세션', '신규방문자', '페이지뷰']:
    train[f'{target}CumSum'] = train[target].cumsum()
    # log하고 rolling mean
    train[target] = log_trans(train[target])
    
    for k in [3,7,14,21]:
        train[f'{target}RollingMean{k}'] =  (train[target].rolling(k).mean())

    train[f'{target}RollingStd21'] =  (train[target].rolling(21).std().round(0))
    train[f'{target}DaysSince10000'] = (train[f'{target}CumSum'] > 10000) * 1
    train[f'{target}DaysSince100000'] = (train[f'{target}CumSum'] > 100000) * 1

    train[f'{target}RollingMeanDiff2w'] = train[f'{target}RollingMean7'] / (train[f'{target}RollingMean14'] + 1) - 1
    train[f'{target}RollingMeanDiff3w'] = train[f'{target}RollingMean7'] / (train[f'{target}RollingMean21'] + 1) - 1


    train[f'{target}CumSum'] = log_trans(train[target].cumsum())
    
    
train = train.dropna()

In [ ]:
def make_data(df, window_size=120):
  # in 180
  input_window = window_size
  # out 61 
  output_window = 61

  window_x = np.zeros((df.shape[0] - (input_window + output_window), input_window, df.shape[1]-1))
  window_y = np.zeros((df.shape[0] - (input_window + output_window), output_window, 4))

  for start in range(df.shape[0] - (input_window + output_window)):
      end = start + input_window    
      window_x[start,:, :] = df.iloc[start : end, 1: ].values
      window_y[start,:, :] = df.iloc[end   : end + output_window, 1: 5].values


  return window_x, window_y

class DatasetWindows(Dataset):
  def __init__(self, df):
    x, y = make_data(df)
    
    self.x = torch.tensor(x, dtype=torch.float32).cuda()
    self.y = torch.tensor(y, dtype=torch.float32).cuda()
    
  def __len__(self):
    return len(self.x)
    
  def __getitem__(self, idx):
    return self.x[idx,...], self.y[idx,...]

In [ ]:
class Embedding(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Embedding, self).__init__()
     
        self.embedding = nn.Sequential(nn.Linear(49, 32),
                                       nn.Dropout(0.8),
                                       nn.ReLU())

    def forward(self, input):
        embedded = self.embedding(input)
        return embedded


class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, batch_size=30):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.embedding = Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output, hidden = self.gru(embedded, hidden)
        # B 1 32
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.batch_size, 1, self.hidden_size).cuda()


class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, batch_size=30):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        # B x 1 x 4
        embedded = embedded
        output = F.relu(embedded)
        output, hidden = self.gru(output, hidden)
        output = self.out(output[0])
        # B x 1 x 4
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.batch_size, 1, self.hidden_size, device=device)



In [ ]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=120, batch_size=30):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(1)
    target_length = target_tensor.size(1)

    # B x 120 x 32
    encoder_outputs = torch.zeros(batch_size, max_length, encoder.hidden_size).cuda()

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[:,ei,:] = encoder_output[:, 0, :]

    # x_n을 input으로
    decoder_input = input_tensor[:4,-1,:]
    # 마지막 hidden state 값을 initial으로 
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        decoder_input = decoder_output.detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[:,di,:])

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=100, plot_every=100, learning_rate=0.001):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer =  torch.optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)

    criterion = nn.NLLLoss()

    for epoch in range(num_epochs):
      for data in dataloader:
        input_tensor, target_tensor = data
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

      if epoch % 100 == 99:
          val_loss = get_val_loss()
          print(f"{epoch+1} Epochs train MSE: {loss.item():1.5f}, ", f"{epoch+1} Epochs val MSE: {val_loss:1.5f}")

In [ ]:
class ModelManager():
  def __init__(self, model_name, df, device='gpu', cv=50):
    super(ModelManager, self).__init__()
    self.cv = cv
    # CV 구현..
    self.models = []
    self.dataloders = []
    
    for i in range(cv):  
      model =  model_name()
      if device == 'gpu':
        model =  model.cuda()
      self.models.append(model)

      cv_set = df
      dataset = DatasetWindows(cv_set)
      self.dataloders.append(DataLoader(dataset, batch_size=30,  num_workers=0, pin_memory=False,
                                        shuffle=True))
    
  def fit(self, num_epochs=500, lr=1e-2 ,log=False, val_set=None, train_set=None):
    
    def get_val_loss():
      val_loss = self.make_val_plot(val_set, train_set, get_loss=True)
      return val_loss

    for i in tqdm(range(self.cv)):  
      # Train model
      model = self.models[i]
      dataloader = self.dataloders[i]
      optimizer = torch.optim.Adam(model.parameters(), lr=lr)
      criterion = nn.MSELoss(size_average = True)

      for epoch in range(num_epochs):
          for data in dataloader:
              x, y = data
              train_pred = model(x)
              loss = criterion(train_pred, y)
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()
          if epoch % 100 == 99:
            if log:
              if val_set.any().any():
                val_loss = get_val_loss()
                print(f"{epoch+1} Epochs train MSE: {loss.item():1.5f}, ", f"{epoch+1} Epochs val MSE: {val_loss:1.5f}")
              else: 
                print(f"{epoch+1} Epochs train MSE: {loss.item():1.5f}")


  @staticmethod
  def inverse_log(x):
    # 32bit 사용시 단위문제 발생..
    return (np.exp(x)-1).astype(np.int64)

  def predict(self, df):

    last_observe = df.iloc[-180:,1:]
    inp_tensor = torch.tensor(last_observe.values, dtype=torch.float32).cuda()
    inp_tensor = inp_tensor.unsqueeze(0)

    model = self.models[0].eval()
    prediction = model(inp_tensor)
    self.models[0] = model.train()

    for i in range(1, self.cv): 
      model = self.models[i].eval()
      prediction += model(inp_tensor)
      self.models[i] = model.train()
    
    prediction /= self.cv
    return  self.inverse_log(prediction.cpu().detach().squeeze().numpy())

  def make_val_plot(self, val_df, train_df, get_loss=False):
    #predict
    pred = self.predict(train_df)
    label = self.inverse_log(val_df)

    def dacon_rmse(true, pred):  
      w0 = 1095.214646
      w1 = 1086.728535
      w2 = 268.070707
      w3 = 24236.194444

      score = (np.sqrt(np.mean(np.square(true[:,0] - pred[:,0]))) / w0 + 
               np.sqrt(np.mean(np.square(true[:,1] - pred[:,1]))) / w1 + 
               np.sqrt(np.mean(np.square(true[:,2] - pred[:,2]))) / w2 + 
               np.sqrt(np.mean(np.square(true[:,3] - pred[:,3]))) / w3  )
      return score

    if get_loss:
      return dacon_rmse(label.values, pred)

    for idx, key in enumerate(val_df.columns):
      plt.plot(figsize=(20,10))
      plt.plot(label.index,pred[:,idx])
      plt.plot(label[key])
      plt.legend(['predict', 'label'])
      plt.show()
      
    loss = dacon_rmse(label.iloc[:,:4].values, pred)
    print('RMSE : ' + str(loss))